# 0.1. imports

In [15]:
import bw2io
import bw2data
import numpy as np
import pandas as pd
# type hints
from bw2io import SingleOutputEcospold2Importer

In [5]:
bw2data.databases

Databases dictionary with 0 objects

# 0.2. file paths

In [32]:
path_dir_datasets_ecoinvent31: str = '/Users/michaelweinold/data/data_raw/ecoinvent_3.1_cutoff_ecoSpold02/datasets'
path_dir_datasets_ecoinvent35: str = '/Users/michaelweinold/data/data_raw/ecoinvent_3.5_cutoff_ecoSpold02/datasets'
path_dir_datasets_ecoinvent38: str = '/Users/michaelweinold/data/data_raw/ecoinvent_3.8_cutoff_ecoSpold02/datasets'

# 1. import data

compare, for instance, https://github.com/brightway-lca/brightway2/blob/35d41cea7080459237986e4babc58c7e93ce5036/notebooks/IO%20-%20importing%20Ecoinvent.ipynb

## 1.1. import `Ecoinvent 3.8`

"Which `bw2io` version to use?"

Compare the [Brightway mailing list](https://brightway.groups.io/g/development/message/154) and the [`bw2io` changelog](https://github.com/brightway-lca/brightway2-io/blob/main/CHANGES.md#085-2021-09-21): use `0.8.7`

In [21]:
bw2data.projects.set_current("ecoinvent38")

In [26]:
importer_ei38: SingleOutputEcospold2Importer = bw2io.SingleOutputEcospold2Importer(
    dirpath = path_dir_datasets_ecoinvent38,
    db_name = 'ei38'
)

Extracting XML data from 19565 datasets
Extracted 19565 datasets in 15.74 seconds


In [27]:
importer_ei38.apply_strategies()

Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows


/opt/homebrew/Caskroom/miniconda/base/envs/bw/lib/python3.10/site-packages/bw2io/strategies/ecospold2.py:246: UserWarning: Skipping migration 'fix-ecoinvent-flows-pre-35' because it isn't installed
  warnings.warn(


Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy: set_lognormal_loc_value
Applying strategy: convert_activity_parameters_to_list
Applying strategy: add_cpc_classification_from_single_reference_product
Applying strategy: delete_none_synonyms
Applied 21 strategies in 2.96 seconds


In [28]:
importer_ei38.statistics()

19565 datasets
629959 exchanges
388452 unlinked exchanges
  Type biosphere: 1018 unique unlinked exchanges


(19565, 629959, 388452)

## 1.2. import `Ecoinvent 3.5`

In [4]:
bw2data.projects.set_current("ecoinvent35")

In [16]:
importer_ei35: SingleOutputEcospold2Importer = bw2io.SingleOutputEcospold2Importer(
    dirpath = path_dir_datasets_ecoinvent35,
    db_name = 'ei35'
)

Extracting XML data from 16022 datasets
Extracted 16022 datasets in 12.14 seconds


In [19]:
importer_ei35.apply_strategies()

Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows


/opt/homebrew/Caskroom/miniconda/base/envs/bw/lib/python3.10/site-packages/bw2io/strategies/ecospold2.py:246: UserWarning: Skipping migration 'fix-ecoinvent-flows-pre-35' because it isn't installed
  warnings.warn(


Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy: set_lognormal_loc_value
Applying strategy: convert_activity_parameters_to_list
Applying strategy: add_cpc_classification_from_single_reference_product
Applying strategy: delete_none_synonyms
Applied 21 strategies in 2.30 seconds


In [20]:
importer_ei35.statistics()

16022 datasets
544735 exchanges
370448 unlinked exchanges
  Type biosphere: 1015 unique unlinked exchanges


(16022, 544735, 370448)

## 1.3. import `Ecoinvent 3.1`

In [30]:
bw2data.projects.set_current("ecoinvent31")

In [36]:
importer_ei31: SingleOutputEcospold2Importer = bw2io.SingleOutputEcospold2Importer(
    dirpath = path_dir_datasets_ecoinvent31,
    db_name = 'ei31'
)

Extracting XML data from 11301 datasets
Extracted 11301 datasets in 13.63 seconds


In [38]:
importer_ei31.apply_strategies()

Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid


/opt/homebrew/Caskroom/miniconda/base/envs/bw/lib/python3.10/site-packages/bw2io/strategies/ecospold2.py:246: UserWarning: Skipping migration 'fix-ecoinvent-flows-pre-35' because it isn't installed
  warnings.warn(


Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy: set_lognormal_loc_value
Applying strategy: convert_activity_parameters_to_list
Applying strategy: add_cpc_classification_from_single_reference_product
Applying strategy: delete_none_synonyms
Applied 21 strategies in 2.10 seconds


In [39]:
importer_ei31.statistics()

11301 datasets
520932 exchanges
267505 unlinked exchanges
  Type biosphere: 929 unique unlinked exchanges


(11301, 520932, 267505)